# Data Preparation and Processing

In [2]:
import xarray as xr
import numpy as np

In [17]:
era5 = xr.open_zarr(
    'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_with_poles_conservative.zarr',
    chunks={'time': 48},
    consolidated=True,
    )
print("Model surface dataset size {} GiB".format(era5.nbytes/(1024**3)))
era5

/Users/amarmesic/miniconda3/envs/mlg/lib/python3.10/site-packages/xarray/core/dataset.py:265: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 48. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


Model surface dataset size 75.83941642194986 GiB


<xarray.Dataset>
Dimensions:                                           (time: 92040,
                                                       longitude: 64,
                                                       latitude: 32, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 -90....
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float64 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    ...                                                ...
    type_of_high_vegetation                           (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    type_of_low_vegetation                            (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    u_component_of_wind                               (time, level, longitude, latitude) float32 dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind                               (time, level, longitude, latitude) float32 dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>
    vertical_velocity                                 (time, level, longitude, latitude) float32 dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>
    wind_speed                                        (time, level, longitude, latitude) float32 dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>

### The dataset is still huge, we shrink it

In [53]:
# Select specific data variables
selected_vars = [
    '10m_u_component_of_wind', '10m_v_component_of_wind', 
    '10m_wind_speed', 
    '2m_temperature', 
    'mean_sea_level_pressure', 
    'surface_pressure', 'toa_incident_solar_radiation_6hr', 
    'total_cloud_cover', 
    'total_column_water_vapour',
    'total_precipitation_6hr']
era5_selected = era5[selected_vars]
print("Model surface dataset size {} GiB".format(era5_selected.nbytes/(1024**3)))
era5_selected

Model surface dataset size 7.022781193256378 GiB


<xarray.Dataset>
Dimensions:                           (time: 92040, longitude: 64, latitude: 32)
Coordinates:
  * latitude                          (latitude) float64 -90.0 -84.19 ... 90.0
  * longitude                         (longitude) float64 0.0 5.625 ... 354.4
  * time                              (time) datetime64[ns] 1959-01-02 ... 20...
Data variables:
    10m_u_component_of_wind           (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind           (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    10m_wind_speed                    (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    2m_temperature                    (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    mean_sea_level_pressure           (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    surface_pressure                  (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation_6hr  (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    total_cloud_cover                 (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    total_column_water_vapour         (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    total_precipitation_6hr           (time, longitude, latitude) float32 dask.array<chunksize=(48, 64, 32), meta=np.ndarray>

In [54]:
# Select a specific range of dimensions
era5_reduced = era5_selected.sel(
    # lat=slice(-90, 90),  # Select latitude range
    # lon=slice(-180, 180),  # Select longitude range
    time=slice('2005-01-01', '2010-01-01')  # Select time range
)
print("Model surface dataset size {} MiB".format(era5_reduced.nbytes/(1024**2)))
era5_reduced

Model surface dataset size 570.9939880371094 MiB


<xarray.Dataset>
Dimensions:                           (time: 7308, longitude: 64, latitude: 32)
Coordinates:
  * latitude                          (latitude) float64 -90.0 -84.19 ... 90.0
  * longitude                         (longitude) float64 0.0 5.625 ... 354.4
  * time                              (time) datetime64[ns] 2005-01-01 ... 20...
Data variables:
    10m_u_component_of_wind           (time, longitude, latitude) float32 dask.array<chunksize=(44, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind           (time, longitude, latitude) float32 dask.array<chunksize=(44, 64, 32), meta=np.ndarray>
    10m_wind_speed                    (time, longitude, latitude) float32 dask.array<chunksize=(44, 64, 32), meta=np.ndarray>
    2m_temperature                    (time, longitude, latitude) float32 dask.array<chunksize=(44, 64, 32), meta=np.ndarray>
    mean_sea_level_pressure           (time, longitude, latitude) float32 dask.array<chunksize=(44, 64, 32), meta=np.ndarray>
    surface_pressure                  (time, longitude, latitude) float32 dask.array<chunksize=(44, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation_6hr  (time, longitude, latitude) float32 dask.array<chunksize=(44, 64, 32), meta=np.ndarray>
    total_cloud_cover                 (time, longitude, latitude) float32 dask.array<chunksize=(44, 64, 32), meta=np.ndarray>
    total_column_water_vapour         (time, longitude, latitude) float32 dask.array<chunksize=(44, 64, 32), meta=np.ndarray>
    total_precipitation_6hr           (time, longitude, latitude) float32 dask.array<chunksize=(44, 64, 32), meta=np.ndarray>

## Final Dataset size: 1827 MiB

In [58]:
has_nans = era5_reduced.isnull().any()
has_nans.to_array().values

array([False, False, False, False, False, False, False, False, False,
       False])

In [50]:
# in case we have nan values, but right now we don't

def fill_na_with_neighbors(da):
    """
    Fill NaN values in the DataArray by averaging their neighbors.
    """
    isnan = np.isnan(da)
    avg_neighbors = np.zeros_like(da)
    count_neighbors = np.zeros_like(da)

    # Shift in all directions and sum up neighbors
    for shift in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        shifted = np.roll(da, shift=shift, axis=(0, 1))
        mask_shifted = np.roll(isnan, shift=shift, axis=(0, 1))
        avg_neighbors += np.where(mask_shifted, 0, shifted)
        count_neighbors += ~mask_shifted

    avg_neighbors = np.where(count_neighbors > 0, avg_neighbors / count_neighbors, np.nan)
    filled = np.where(isnan, avg_neighbors, da)
    return filled

# # Apply the function to all DataArrays in the dataset
# ds_filled = era5_reduced.load().apply(lambda da: xr.apply_ufunc(fill_na_with_neighbors, da))

In [61]:
# Save the reduced dataset to a new file
era5_reduced.to_netcdf('datasets/era5.nc')